In [1]:
import requests
import pandas as pd 
pd.set_option('mode.chained_assignment', None)

In [2]:
df = pd.read_csv("./rawData.csv", index_col=False) 

In [3]:
def setVesselType(number):
    type = '';
    if(number == '6_'):
        type = 'passenger';
    if(number == '7_'):
        type = 'cargo';
    if(number == '8_'):
        type = 'tanker';
    if(number == '3_'):
        type = 'tug';
    if(number == '0_'):
        type = 'undefined';
    if(number == '9_'):
        type = 'yacht';
    if(number == '4_'):
        type = 'high speed';
    if(number == '10'):
        type = 'fishing';
    if(number == '13'):
        type = 'navigation aid';
    return type;

In [4]:
#clean VesselType
for i in df.index:
    number = df['VesselType'][i][11:13]
    df['VesselType'][i] = setVesselType(number)

In [5]:
print(df)
df.to_csv("preparedData.csv", index=False)

        Unnamed: 0      Event              Time          Port  \
0                0  Departure  2019-12-08 20:18  PORT DICKSON   
1                1  Departure  2019-12-08 20:18   PORT JEROME   
2                2  Departure  2019-12-08 20:18     STAVANGER   
3                3  Departure  2019-12-08 20:18          TARS   
4                4  Departure  2019-12-08 20:18       GOSPORT   
...            ...        ...               ...           ...   
249995      249995    Arrival  2019-12-05 14:16    WHITSTABLE   
249996      249996    Arrival  2019-12-05 14:16        BOGNES   
249997      249997    Arrival  2019-12-05 14:16     ROZENBURG   
249998      249998    Arrival  2019-12-05 14:16     STORNOWAY   
249999      249999    Arrival  2019-12-05 14:16     ROTTERDAM   

                           Vessel VesselType  
0                ORKIM POWER [MY]     tanker  
1                     VAUBAN [FR]  undefined  
2                      TEDNO [NO]  passenger  
3                    LOLLAND [D

In [6]:
#create set of vessels
vessels = set(df.Vessel)

In [7]:
#create new df for linked values (arrival and departure)
timetable = pd.DataFrame([], columns = ['DeparturePort', 'DepartureTime', 'ArrivalPort', 'ArrivalTime', 'Vessel', 'VesselType'])

In [8]:
#vessel filter and combine arrival and deaprture
for v in vessels:
    allEntriesOfVessel = df[df['Vessel'].isin([v])]
    depFirst = False
    for index, row in allEntriesOfVessel[::-1].iterrows():
        arrivalPort, arrivalTime = "", ""
        
        if(allEntriesOfVessel.Event[index] == "Departure"):
            departurePort = (allEntriesOfVessel.Port[index])
            departureTime = (allEntriesOfVessel.Time[index])
            vessel = (allEntriesOfVessel.Vessel[index])
            vesselType = (allEntriesOfVessel.VesselType[index])
            depFirst = True
        if(allEntriesOfVessel.Event[index] == "Arrival" and depFirst):
            arrivalPort = (allEntriesOfVessel.Port[index])
            arrivalTime = (allEntriesOfVessel.Time[index])
            depFirst = False

            if(departurePort != "" or departureTime != "" or vessel != "" or vesselType != "" or arrivalPort != "" or arrivalTime!= ""):
                newEntry = pd.DataFrame([[departurePort, departureTime, arrivalPort, arrivalTime, vessel, vesselType]], columns = ['DeparturePort', 'DepartureTime', 'ArrivalPort', 'ArrivalTime', 'Vessel', 'VesselType'])
                timetable = timetable.append(newEntry, ignore_index = True)
            

In [9]:
#drop all rows that have any NaN values
timetable = timetable.dropna()

In [10]:
print(timetable)
timetable.to_csv("timetable.csv")

       DeparturePort     DepartureTime ArrivalPort       ArrivalTime  \
0           GUSTAVIA  2019-12-05 19:59    GUSTAVIA  2019-12-06 09:56   
1           GUSTAVIA  2019-12-06 12:52    GUSTAVIA  2019-12-07 10:06   
2      FREEPORT (TX)  2019-12-05 20:10     HOUSTON  2019-12-06 07:59   
3              KAWAU  2019-12-06 20:05       KAWAU  2019-12-07 03:08   
4              KAWAU  2019-12-07 22:17       KAWAU  2019-12-08 00:26   
...              ...               ...         ...               ...   
98287         TUAPSE  2019-12-07 04:39      TUAPSE  2019-12-07 04:42   
98288         TUAPSE  2019-12-07 07:55      TUAPSE  2019-12-07 08:51   
98289         TUAPSE  2019-12-07 12:13      TUAPSE  2019-12-07 12:23   
98290         TUAPSE  2019-12-07 18:53      TUAPSE  2019-12-07 19:43   
98291         TUAPSE  2019-12-08 13:50      TUAPSE  2019-12-08 14:09   

                      Vessel VesselType  
0               HIRUNDO [BL]      cargo  
1               HIRUNDO [BL]      cargo  
2        